In [11]:
import pandas as pd
import numpy as np
import hypernetx as hnx
import mcopt.cot as cot

In [12]:
graph1 = pd.read_csv('data/graph1.csv')

graph2 = pd.read_csv('data/graph2.csv')

graph1

,Cell ID,Cell Type,DestinationId,NumberOfCriticalPointsOnBoundary,Point Index 0,Point Index 1,SeparatrixFunctionDifference,SeparatrixFunctionMaximum,SeparatrixFunctionMinimum,SeparatrixId,SeparatrixType,SourceId
0,0,3,0,2,0,1,0.565527,-0.283251,-0.848778,0,0,69
1,1,3,0,2,1,2,0.565527,-0.283251,-0.848778,0,0,69
2,2,3,0,2,2,3,0.565527,-0.283251,-0.848778,0,0,69
3,3,3,0,2,3,4,0.565527,-0.283251,-0.848778,0,0,69
4,4,3,0,2,4,5,0.565527,-0.283251,-0.848778,0,0,69
...,...,...,...,...,...,...,...,...,...,...,...,...
1747,1747,3,9999,2,1794,1795,0.572192,-0.258832,-0.831024,47,0,29558
1748,1748,3,9999,2,1795,1796,0.572192,-0.258832,-0.831024,47,0,29558
1749,1749,3,9999,2,1796,1797,0.572192,-0.258832,-0.831024,47,0,29558
1750,1750,3,9999,2,1797,1798,0.572192,-0.258832,-0.831024,47,0,29558


In [13]:
def gen_hypergraph(csv):
  edges = set()

  for _, row in csv.iterrows():
    edges.add((int(row['SourceId']), int(row['DestinationId'])))

  h_dict = {}

  for i, edge in enumerate(edges):
    h_dict['e' + str(i)] = list(edge)

  return hnx.Hypergraph(h_dict)

h1 = gen_hypergraph(graph1)
h2 = gen_hypergraph(graph2)

h2.shape

(54, 65)

In [14]:
u1 = np.repeat(1/h1.shape[0], h1.shape[0])
u2 = np.repeat(1/h2.shape[0], h2.shape[0])

v1 = np.repeat(1/h1.shape[1], h1.shape[1])
v2 = np.repeat(1/h2.shape[1], h2.shape[1])

In [15]:
def get_omega_no_weight(hgraph):
  # w_ij = 1 if node in the hyperedge, otherwise w_ij = 0
  num_nodes, num_edges = hgraph.shape
  h_dict = hgraph.incidence_dict
  
  h_edges = list(hgraph.edges)
  h_nodes = list(hgraph.nodes)
  h_edges.sort()
  h_nodes.sort()
  edges_id2idx = {}
  nodes_id2idx = {}
  for i in range(len(h_edges)):
    edges_id2idx[h_edges[i]] = i
  for i in range(len(h_nodes)):
    nodes_id2idx[h_nodes[i]] = i

  w = np.zeros((num_nodes, num_edges))
  for edge in h_dict:
    edge_idx = edges_id2idx[edge]
    nodes_list = h_dict[edge]
    for node in nodes_list:
      node_idx = nodes_id2idx[node]
      w[node_idx, edge_idx] = 1
  return w

In [16]:
w1 = get_omega_no_weight(h1)
w2 = get_omega_no_weight(h2)

In [17]:
Ts, Tv, cost, log=cot.cot_numpy(w1, w2, w1=u1, w2=u2, v1=v1, v2=v2, niter=100,log=True,verbose = False)

In [18]:
cost

0.03244123931623931

In [19]:
print('Vertex Matching from h1 to h2')

for fro, to in enumerate(np.argmax(Ts, axis=1)):
  print(str(fro) + ' -> ' + str(to))

print('Hyperedge Matching from h1 to h2')

for fro, to in enumerate(np.argmax(Tv, axis=1)):
  print(str(fro) + ' -> ' + str(to))

Vertex Matching from h1 to h2
0 -> 33
1 -> 35
2 -> 13
3 -> 0
4 -> 3
5 -> 4
6 -> 37
7 -> 28
8 -> 20
9 -> 30
10 -> 5
11 -> 48
12 -> 40
13 -> 53
14 -> 18
15 -> 10
16 -> 9
17 -> 1
18 -> 8
19 -> 50
20 -> 47
21 -> 43
22 -> 12
23 -> 23
24 -> 7
25 -> 31
26 -> 46
27 -> 45
28 -> 2
29 -> 42
30 -> 14
31 -> 6
32 -> 32
33 -> 26
34 -> 21
35 -> 25
36 -> 52
37 -> 38
38 -> 49
39 -> 34
Hyperedge Matching from h1 to h2
0 -> 13
1 -> 32
2 -> 61
3 -> 41
4 -> 15
5 -> 3
6 -> 56
7 -> 63
8 -> 6
9 -> 28
10 -> 2
11 -> 0
12 -> 59
13 -> 33
14 -> 29
15 -> 23
16 -> 11
17 -> 42
18 -> 57
19 -> 51
20 -> 10
21 -> 38
22 -> 25
23 -> 24
24 -> 64
25 -> 54
26 -> 62
27 -> 27
28 -> 46
29 -> 8
30 -> 21
31 -> 43
32 -> 19
33 -> 58
34 -> 20
35 -> 22
36 -> 34
37 -> 36
38 -> 44
39 -> 9
40 -> 18
41 -> 12
42 -> 17
43 -> 49
44 -> 4
45 -> 5
46 -> 39
47 -> 53
